# VQE in DFT with PsiEmbed and Qiskit

Here we define the inputs as required by PsiEmbed. Note that we'll follow the logic of `embedding_module/run_open_shell`.

We can think of this procedure are requiring three steps:

1. Pre-embedding

    Here we define the problem, and run a low-level calculation of the whole system. From this we obtain the pre-embedded density matrices $\gamma^A$ and $\gamma^B$

    We then define the level-shift projector $P$ and embedding potential $V_{emb}$.

2. Embedding

    Using $V_{emb}$ we run a high-level method simulation of the active region to get the embedded density matrix $\gamma^A_{emb}$.

    We calculate the correction term $tr[V_{emb}(\gamma^A_{emb}-\gamma^A)]$

3. Post-embedding

    Finally we calculate the embedded energy, by removing $V_{emb}$ from the Hamiltonian, and using density matrix $\gamma^A_{emb}$.

    The total energy is then given by: $E = E[\gamma^A_{emb}] + E[\gamma^B] + g[\gamma^A, \gamma^B] + E_{nuclear} + tr[V_{emb}(\gamma^A_{emb}-\gamma^A)]$

# 0. Set Parameters

First we'll set the parameters 

In [1]:
from copy import copy
import numpy as np
from scipy import linalg
from typing import Dict

ethane = """
C      0.7516     -0.0225     -0.0209
C     -0.7516      0.0225      0.0209
H      1.1851     -0.0039      0.9875
H      1.1669      0.8330     -0.5693
H      1.1155     -0.9329     -0.5145
H     -1.1669     -0.8334      0.5687  
H     -1.1157      0.9326      0.5151
H     -1.1850      0.0044     -0.9875
    """

ethylene = """
C    3.402   0.773  -9.252
H    2.933  -0.150  -9.521
H    2.837   1.682  -9.258
C    4.697   0.791  -8.909
H    5.262  -0.118  -8.904
H    5.167   1.714  -8.641
    """

methanol = """
O     -0.6582     -0.0067      0.1730 
H     -1.1326     -0.0311     -0.6482
C      0.7031      0.0083     -0.1305
H      1.2001      0.0363      0.8431
H      0.9877      0.8943     -0.7114
H      1.0155     -0.8918     -0.6742
  """
# H     -1.1326     -0.0311     -0.6482 <--- this goes with the oxygen


formaldehyde = """
C      0.5979      0.0151      0.0688
H      1.0686     -0.1411      1.0408
H      1.2687      0.2002     -0.7717
O     -0.5960     -0.0151     -0.0686
  """

water = """
O          0.00000        0.000      0.115
H          0.00000        0.754     -0.459
H          0.00000       -0.754     -0.459
    """

h_peroxide = """
O          0.00000        0.00000        0.00000
O          1.00000        0.00000        0.00000
H          0.00000        0.50000        0.00000
H          1.00000       -0.50000        0.00000
"""

LiH = """
Li 0.0 0.0 0.0
H  0.0 0.0 1.541
"""

Li2 = """
Li 0.0 0.0 0.0
Li  0.0 0.0 2.627
"""

glycine = """
O     1.916  -0.760  -0.740 
H    -1.736  -0.252  -0.592
N    -1.476   0.232   0.252 
C    -0.012   0.296   0.348 
C     0.596  -0.652  -0.648 
O    -0.124  -1.320  -1.368 
H     0.292   0.020   1.364 
H     0.320   1.320   0.132 
H    -1.636  -0.236   1.132 
H    -1.916   1.136   0.272
  """

pyridine = """
  N      0.6816      1.1960      0.0000
  C      1.3603      0.0256      0.0000
  C      0.6971     -1.2020      0.0000
  C     -0.6944     -1.2184      0.0000
  C     -1.3895     -0.0129      0.0000
  C     -0.6712      1.1834      0.0000
  H      2.4530      0.1083      0.0000
  H      1.2665     -2.1365      0.0000
  H     -1.2365     -2.1696      0.0000
  H     -2.4837      0.0011      0.0000
  H     -1.1569      2.1657      0.0000
"""

options = {}
options['geometry'] = water
options['n_active_atoms'] = 2 # number of active atoms (first n atoms in the geometry string)

run_fci = True
run_vqe = False

options['basis'] = 'STO-3G'#'6-31G*' # basis set 
options['low_level'] = 'LDA_C_VWN' # level of theory of the environment 
options['high_level'] = 'mp2' # level of theory of the embedded system
options['low_level_reference'] = 'rhf'
options['high_level_reference'] = 'rhf'
options['package'] = 'pyscf'
options['e_convergence'] = 1e-10

def fill_defaults(keywords) -> Dict[str, str]:
    "Fills in the default keyword values"
    # Default keywords
    default_keywords = {}
    default_keywords["package"] = "pyscf"
    default_keywords["num_threads"] = 1
    default_keywords["memory"] = 1000
    default_keywords["charge"] = 0
    default_keywords["multiplicity"] = 1
    default_keywords["low_level_reference"] = "rhf"
    default_keywords["high_level_reference"] = "rhf"
    default_keywords["partition_method"] = "spade"
    default_keywords["e_convergence"] = 1e-6
    default_keywords["d_convergence"] = 1e-6
    default_keywords["eri"] = "df"
    default_keywords["ints_tolerance"] = 1e-10
    default_keywords["driver_output"] = "output.dat"
    default_keywords["embedding_output"] = "embedding.log"
    default_keywords["operator"] = "F"
    default_keywords["level_shift"] = 1.0e6
    default_keywords["low_level_damping_percentage"] = 0
    default_keywords["high_level_damping_percentage"] = 0
    default_keywords["low_level_soscf"] = "False"
    default_keywords["high_level_soscf"] = "False"
    default_keywords["molden"] = False
    default_keywords["print_level"] = 1
    default_keywords["cc_type"] = "df"
    default_keywords["save_embedding_potential"] = False
    default_keywords["save_embedded_h_core"] = False
    default_keywords["save_embedded_orbitals"] = False
    default_keywords["run_high_level"] = True

    # Checking if the necessary keywords have been defined
    required = ["low_level", "high_level", "basis", "n_active_atoms"]
    if not all([key in keywords for key in required]):
        raise KeyError("You must include keys for each of:\n%s", required)
    # assert 'low_level' in keywords, ('\n Choose level of theory',
    #                                 'for the environment')
    # assert 'high_level' in keywords, ('\n Choose level of theory',
    #                                 'for the active region')
    # assert 'basis' in keywords, '\n Choose a basis set'
    # assert 'n_active_atoms' in keywords, ('\n Provide the number of active',
    #     'atoms, which the first atoms in your coordinates string')

    for key in default_keywords.keys():
        if key not in keywords:
            keywords[key] = default_keywords[key]

    if "n_cl_shell" in keywords and "virtual_projection_basis" not in keywords:
        keywords["virtual_projection_basis"] = keywords["basis"]

    return keywords

keywords = fill_defaults(options)

#run_closed_shell(keywords)

# 1. Low-level whole system calculation

The first step is to run a mean field caluclation of the whole system.    

We also run a high-precision method, Full Configuration Interaction or Coupled Cluster Single Double to determine a reference value.

In [2]:
from pyscf import gto, scf, cc, ao2mo, fci

mol = gto.Mole(atom=keywords['geometry'], basis=keywords['basis'], charge=0).build()

ks = scf.RKS(mol)
ks.conv_tol = keywords["e_convergence"]
ks.xc = keywords["low_level"]
e_initial = ks.kernel()

# Store the initial value of h core as this is needed later,
# but is overwritten

initial_h_core = ks.get_hcore()

# Calulate a reference value
mol_copy = copy(mol)
hf = mol_copy.RHF().run()
if run_fci:
    ref_fci = fci.FCI(hf)
    ref_fci.conv_tol = keywords["e_convergence"]
    fci_result = ref_fci.kernel()

    # This DOES have nuclear energy included!
    expected_energy = fci_result[0]

else:
    ref_cc = cc.CCSD(hf)
    ref_cc.conv_tol = keywords["e_convergence"]
    cc_result = ref_cc.kernel()

    expected_energy = hf.energy_tot() + cc_result[0]

print(f"{e_initial}, {expected_energy}")

converged SCF energy = -66.5930189922302
converged SCF energy = -74.9609996030856
-66.59301899223016, -75.00912605536345


# 2. Orbital localization
Find the orbitals of the active space and environment, using SPADE.

In [3]:
n_occupied_mos = np.count_nonzero(ks.mo_occ == 2)
occupied_orbitals = ks.mo_coeff[:, :n_occupied_mos]
n_act_aos = mol.aoslice_by_atom()[keywords['n_active_atoms']-1][-1]

# Rotate orbitals so that they're orthogonal
ao_overlap = ks.get_ovlp()
rotated_orbitals = linalg.fractional_matrix_power(ao_overlap, 0.5) @ occupied_orbitals
_, sigma, right_vectors = linalg.svd(rotated_orbitals[:n_act_aos, :])

#n_act_mos, n_env_mos = embed.orbital_partition(sigma)
value_diffs = sigma[:-1]-sigma[1:]
n_act_mos = np.argmax(value_diffs) + 1
n_env_mos = n_occupied_mos - n_act_mos

# Defining active and environment orbitals and 
act_orbitals = occupied_orbitals @ right_vectors.T[:, :n_act_mos]
env_orbitals = occupied_orbitals @ right_vectors.T[:, n_act_mos:n_act_mos+n_env_mos]
act_density = 2.0 * act_orbitals @ act_orbitals.T
env_density = 2.0 * env_orbitals @ env_orbitals.T

print(f"{n_act_mos}, {n_env_mos}")

4, 1


# 3. Calculate the cross subsytem terms

In [4]:
# Retrieving the subsytem energy terms and potential matrices

# The function called looks like this
def closed_shell_subsystem(scf, density):
        #It seems that PySCF lumps J and K in the J array 
        j = ks.get_j(dm = density)
        k = np.zeros(np.shape(j))
        two_e_term =  ks.get_veff(ks.mol, density)
        e_xc = two_e_term.exc
        v_xc = two_e_term - j

        # Energy
        e = np.einsum("ij,ij", density, ks.get_hcore() + j/2) + e_xc
        return e, e_xc, j, k, v_xc

e_act, e_xc_act, j_act, k_act, v_xc_act = (
    closed_shell_subsystem(scf, act_density))
e_env, e_xc_env, j_env, k_env, v_xc_env = (
    closed_shell_subsystem(scf, env_density))

# Computing cross subsystem terms
# Note that the matrix dot product is equivalent to the trace.
j_cross = 0.5 * (np.einsum("ij,ij",act_density, j_env)
        + np.einsum("ij,ij", env_density, j_act))

k_cross = 0.0

xc_cross = ks.get_veff().exc - e_xc_act - e_xc_env
two_e_cross = j_cross + k_cross + xc_cross
print(f"{e_act}, {e_env}, {two_e_cross}")

-74.25684379828833, -11.33703574566101, 9.715146330041323


# 4. Define $V_{emb}$

We can now define the projector used to orthogonalise the Molecular and Atomic orbitals. From this we calculate the embedding potential.

$P_{\alpha, \beta} = S\gamma^BS$

From this we can now also define the embedding potential.

$V_{emb} = g[\gamma^A, \gamma^B] - g[\gamma^A] + \mu P$

In [5]:
# Define the mu-projector
projector = keywords['level_shift'] * (ks.get_ovlp() @ env_density
    @ ks.get_ovlp())

v_xc_total = ks.get_veff() - ks.get_j()

# Defining the embedded core Hamiltonian
v_emb = (j_env + v_xc_total - v_xc_act + projector)

print(f"{keywords['level_shift']}")

1000000.0


# 5A Run HF of full system with $V_{emb}$ to get $\gamma^A_{emb}$

Here, PsiEmbed gives us the option to stop, outputting values for calculation by other means.

To continue, we run the mean field method, but with the embedding potentials as calulated.

In [6]:
embedded_scf = scf.RHF(mol_copy)
embedded_scf.conv_tol = keywords["e_convergence"]
embedded_scf.mol.nelectron = 2*n_act_mos

h_core = ks.get_hcore()

embedded_scf.get_hcore = lambda *args, **kwargs: h_core + v_emb

embedded_scf.kernel()

embedded_occ_orbs = embedded_scf.mo_coeff[:, embedded_scf.mo_occ>0]
embedded_density = 2*embedded_occ_orbs @ embedded_occ_orbs.T

e_emb = embedded_scf.energy_elec(dm=embedded_density, vhf=embedded_scf.get_veff())[0]

print(f"{e_emb}")

converged SCF energy = -62.7025386383342
-71.98825286001207


Overwritten attributes  get_hcore  of <class 'pyscf.scf.hf.RHF'>
/Users/lex/anaconda3/envs/Embed/lib/python3.7/site-packages/pyscf/gto/mole.py:1089: UserWarning: Function mol.dumps drops attribute _nelectron because it is not JSON-serializable
  warnings.warn(msg)


# 6 Calculate correction term

In [7]:
# Compute the correction
# - There are two versions used for different embeddings
dm_correction = np.einsum("ij,ij", v_emb, embedded_density-act_density)
wf_correction = np.einsum("ij,ij", act_density, v_emb)

print(f"{wf_correction}, {dm_correction}")

9.723526901565492, 0.0987452587523876


# 7 Calculate $E[\gamma^A_{emb}]$

## Classical Method - CCSD

In [8]:
from scipy.linalg import LinAlgError

# Can use either of these methods 
# This needs to change if we're not using PySCFEmbed
# The j and k matrices are defined differently in PySCF and Psi4

embedded_scf.get_hcore = lambda *args, **kwargs: h_core + v_emb

e_wf_act = embedded_scf.energy_elec(dm=embedded_density, vhf=embedded_scf.get_veff())[0]
e_act_emb_explicit = np.einsum("ij,ij", embedded_density,  initial_h_core + 0.5 * embedded_scf.get_j() - 0.25 * embedded_scf.get_k())

print(f"E_HF = {e_wf_act}")
print(f"Difference between HF methods: {e_wf_act - e_act_emb_explicit}")
print(f"Difference between HF methods + corrections: {(e_wf_act - wf_correction) - (e_act_emb_explicit + dm_correction)}")

try:
    # Run CCSD as WF method
    ccsd = cc.CCSD(embedded_scf)
    ccsd.conv_tol = keywords["e_convergence"]

    # Set which orbitals are to be frozen
    # The environment orbitals energies have been increased by the projector
    # so they are now at the end of the list, as orbitals are ordered by energy
    shift = mol.nao - n_env_mos
    fos = [i for i in range(shift, mol.nao)]
    ccsd.frozen = fos
    ccsd.run()
    correlation = ccsd.e_corr
    e_wf_act += correlation
    
except LinAlgError as e:
    print(e)
    print("Use the HF energy")
    pass

f"{e_wf_act}"

E_HF = -71.98825286001207
Difference between HF methods: 9.822272160971437
Difference between HF methods + corrections: 6.535572083521402e-10
E(CCSD) = -62.72682912811824  E_corr = -0.0242904897839919


'-72.01254334979606'

## Quantum Method - VQE

### Set active space

Before setting up the VQE we'll get the active and occupied indices - this will be how we reduce the number of qubits needed.

Active indices include: Occupied MOs assigned to the active region + All unoccupied MOs

In [9]:
from openfermion.ops.representations import InteractionOperator, get_active_space_integrals
from openfermion.linalg import eigenspectrum, expectation
from openfermion.transforms import jordan_wigner

occupied_indices = embedded_scf.mo_occ == 2
occupied_indices = [i for i, val in enumerate(occupied_indices) if val]

# Try using only active MOs
active_indices = [i for i in range(len(embedded_scf.mo_occ)-n_env_mos)]
    

print(f"{len(embedded_scf.mo_occ)}, {embedded_scf.mo_energy}")
print(f"{active_indices} \n {occupied_indices}")

7, [-2.03543356e+01 -1.07107665e+00 -5.14144838e-01 -4.12215748e-01
  6.38010851e-01  8.62267933e-01  1.99999995e+06]
[0, 1, 2, 3, 4, 5] 
 [0, 1, 2, 3]


### Get Qubit Hamiltonian
Now we'll calculate the one and two body integrals to create a fermionic Hamiltonian.

We then cut out the contribution from the environment orbitals. (Note the reduction in the shape of the integrals!)

In [10]:
from functools import reduce
from pyscf import ao2mo

temp_scf = embedded_scf
temp_mol = mol

# We only want the terms related to the active space
coeff = temp_scf.mo_coeff[:, active_indices]

n_orbs = len(active_indices)

# hamiltonian here includes Vemb
one_body_integrals = coeff.T @ temp_scf.get_hcore() @ coeff

temp_mol.incore_anyway == True

# Get two electron integrals in compressed format.
two_body_compressed = ao2mo.kernel(temp_mol, coeff)

two_body_integrals = ao2mo.restore(1, two_body_compressed, n_orbs)

# Openfermion uses pysicist notation whereas pyscf uses chemists
two_body_integrals = np.asarray(two_body_integrals.transpose(0, 2, 3, 1), order='C')

print(f"{one_body_integrals.shape}, {two_body_integrals.shape}")


(6, 6), (6, 6, 6, 6)


We have to transform this to spin-orbital coefficients to make a qubit operator in terms of the Pauli basis.

Then we can find the ground state either by diagonalising the Hamiltonian directly, or by running the VQE algorithm.

In [11]:
print('active inds:',active_indices)
print('occupied inds:', occupied_indices)

active inds: [0, 1, 2, 3, 4, 5]
occupied inds: [0, 1, 2, 3]


In [12]:
from openfermion.chem.molecular_data import spinorb_from_spatial

core_constant=0
one_body_coefficients, two_body_coefficients = spinorb_from_spatial(one_body_integrals, two_body_integrals)
one_body_coefficients.shape

(12, 12)

In [13]:
# active space approximation
active_space_approx = False # <-- flag on whether to use approximation (cell doesn't run if FALSE)


from openfermion.ops.representations import get_active_space_integrals

### can make problem easier by using active space approximation!
### This freezes certain orbitals (one can classical calculate contribution which
### gives a constant correction... this reduces number of qubits required)

# note a user can change the variable:
# reduced_active_inds <-- which indices should be considered active in a calculation (must be subset of active_indices)
# we automate this by using terms_above_occupied_inds and freeze_n_low_lying

# lower number gives more approximation (= less qubits required BUT higher errors due to approximation)
terms_above_occupied_inds= 2 # number of virtual terms included

## freeze first n low lying occupied MOs
freeze_n_low_lying  = 1  # 0 =no occ orbs frozen, 1 = first orbital frozen, 2 = first two orbitals frozen (etc)


if terms_above_occupied_inds > len(active_indices)-len(occupied_indices):
    print(f'can only shift by max: {len(active_indices)-len(occupied_indices)}')
    raise ValueError(f'cannot include {terms_above_occupied_inds} MOs above occ_inds: {occupied_indices} when act_inds are active inds: {active_indices}')

if freeze_n_low_lying>len(occupied_indices)-1:
    print(f'should not freeze all occupied orbitals')
    print(f'freeze_n_low_lying should be 0<= freeze_n_low_lying <{len(occupied_indices)}')
    raise ValueError('n in freeze_n_low_lying too large (more than number of occ orbitals)')


if active_space_approx is True:
    
    # manual choice of indices to include in SCF calc (active space approx used for other inds)
    occ_used = occupied_indices[freeze_n_low_lying:]
    reduced_active_inds = [*occ_used, *list(range(max(occupied_indices)+1, 1+max(occupied_indices)+terms_above_occupied_inds))]
    # note we select the occupied indices + some number of orbs above the occupied indices
    
    print('reduced active indices:', reduced_active_inds)
    print('rather than:', active_indices)
    print(f'reduced orb count: {len(active_indices)-len(reduced_active_inds)}')
    
    core_constant, one_body_ints_reduced, two_body_ints_reduced = get_active_space_integrals(one_body_integrals, 
                                                                        two_body_integrals,
                                                                        occupied_indices=occ_used,
                                                                        active_indices=reduced_active_inds)

    one_body_coefficients, two_body_coefficients = spinorb_from_spatial(one_body_ints_reduced, two_body_ints_reduced)
    print(f'core constant to included (given by frozen orbs): {core_constant}')
    
    # compare to cell before... should be smaller!
    print(one_body_coefficients.shape)
    

In [14]:
print(f'number of qubits needed for VQE will be: {one_body_coefficients.shape[0]} \n')

if one_body_coefficients.shape[0]>15:
    print('*'*8 +' WARNING ' + '*'*8)
    print('LARGE number of qubits! Diagonlization (and VQE) will be hard')
    print('consider using active space appoximation to freeze certain orbitals!')

number of qubits needed for VQE will be: 12 



In [15]:
## make sure device notebook is running on can deal with number of qubits defined above!
## aka matrix diagonalization and multiplications

In [16]:
molecular_hamiltonian = InteractionOperator(core_constant, #<-- core constant added here (0 if no active space approx)
                                            one_body_coefficients,
                                            0.5 * two_body_coefficients,)

Qubit_Hamiltonian = jordan_wigner(molecular_hamiltonian)

e_vqe_act = eigenspectrum(molecular_hamiltonian)[0]
print(f"{e_vqe_act}")

-72.01256812072756


### Add up parts without running VQE

e_act_emb : $\epsilon[\gamma^A_{emb}]$
>energy of the embedded region

e_env : $E[\gamma^B]$ 
>energy of the environment

two_e_cross : $g[\gamma^A, \gamma^B]$
>non-additive two electron term

embed.nre
>The Coulomb energy from nuclear repulsion.

correction : $tr[(\gamma^A_{emb} - \gamma^A)(h^{A in B} - h)]$ (or $tr[\gamma^A(h^{A in B} - h)]$ )
> Correction for embedding

In [17]:
e_nuc = mol.energy_nuc()

e_vqe_emb = e_vqe_act + e_env + two_e_cross + e_nuc - wf_correction 
e_wf_emb = e_wf_act + e_env + two_e_cross + e_nuc - wf_correction 
print("Component contributions")
print(f"{e_vqe_act}, {e_wf_act}")
print(f"{e_env}, {two_e_cross}, {wf_correction}, {dm_correction}, {e_nuc}")
print(f'H_emb described using {one_body_coefficients.shape[0]} qubits')
print(f'active space approx used: {active_space_approx} \n')
# Print out the final value.
print(f"FCI Energy:\t\t\t\t{expected_energy:.4f}") 
print(f"DFT Energy:\t\t\t\t{e_initial:.4f}")
print(f"Error:\t\t\t\t\t{(expected_energy-e_initial)*100/expected_energy:.2f}%")
print(f"WF Embedding Energy:\t\t\t{e_wf_emb:.4f}")
print(f"Error:\t\t\t\t\t{(expected_energy-e_wf_emb)*100/expected_energy:.2f}%")
print(f"VQE H_emb min eig (via lin alg):\t{e_vqe_emb:.4f}")
print(f"Error:\t\t\t\t\t{(expected_energy-e_vqe_emb)*100/expected_energy:.2f}%")

Component contributions
-72.01256812072756, -72.01254334979606
-11.33703574566101, 9.715146330041323, 9.723526901565492, 0.0987452587523876, 9.285714221677825
H_emb described using 12 qubits
active space approx used: False 

FCI Energy:				-75.0091
DFT Energy:				-66.5930
Error:					11.22%
WF Embedding Energy:			-74.0722
Error:					1.25%
VQE H_emb min eig (via lin alg):	-74.0723
Error:					1.25%


### Pennylane VQE

Taking the qubit operator we defined with open fermion, we can perform VQE using the CCSD ansatz.

In [18]:
import pennylane as qml
from pennylane import qchem
from pennylane.templates import UCCSD, ParticleConservingU2
from functools import partial

h = qchem.convert_observable(Qubit_Hamiltonian)

qubits = len(h.wires)
print(f"{qubits}")

electrons = n_act_mos*2

ref_state = qchem.hf_state(electrons, qubits)

# Generate single and double excitations
singles, doubles = qchem.excitations(electrons, qubits)

# Map excitations to the wires the UCCSD circuit will act on
s_wires, d_wires = qchem.excitations_to_wires(singles, doubles)

# Define the device
#dev = qml.device('default.qubit', wires=qubits)
dev = qml.device('default.qubit', wires=qubits)


# Define the UCCSD ansatz
#ansatz = partial(UCCSD, init_state=ref_state, s_wires=s_wires, d_wires=d_wires)
ansatz = partial(ParticleConservingU2, init_state=ref_state)

# Define the cost function
cost_fn = qml.ExpvalCost(ansatz, h, dev)

opt = qml.GradientDescentOptimizer(stepsize=1)

# Compute the expectation value of 'h' for given set of parameters 'params'
layers = 1


12


In [19]:
#UCCSD Params
#params = np.random.normal(0, np.pi, len(singles) + len(doubles))

params = qml.init.particle_conserving_u2_normal(layers, qubits)
explicit_e_vqe_act = None

if run_vqe:
    max_iterations = 100
    conv_tol = keywords["e_convergence"]
    energy = [cost_fn(params)]
    param_vals = [params]

    print("Beginning Iterations")
    print(f"Step = 0,  Energy = {energy[-1]:.8f} Ha")
    for n in range(max_iterations):
        params, prev_energy = opt.step_and_cost(cost_fn, params)

        energy.append(cost_fn(params))
        param_vals.append(params)

        conv = np.abs(energy[-1] - prev_energy)

        # if n % 2 == 0:
        print(f"Step = {n+1},  Energy = {energy[-1]:.8f} Ha")

        if conv <= conv_tol and conv != 0:
            break

    explicit_e_vqe_act = energy[-1]

    print("\n" f"Final value of the ground-state energy = {explicit_e_vqe_act:.8f} Ha")
    print("\n" f"Optimal value of the circuit parameter = {param_vals[-1]}")

# 8. Add all the parts up with VQE

e_act_emb : $\epsilon[\gamma^A_{emb}]$
>energy of the embedded region

e_env : $E[\gamma^B]$ 
>energy of the environment

two_e_cross : $g[\gamma^A, \gamma^B]$
>non-additive two electron term

embed.nre
>The Coulomb energy from nuclear repulsion.

correction : $tr[(\gamma^A_{emb} - \gamma^A)(h^{A in B} - h)]$ (or $tr[\gamma^A(h^{A in B} - h)]$ )
> Correction for embedding

In [20]:
e_nuc = mol.energy_nuc()

e_vqe_emb = e_vqe_act + e_env + two_e_cross + e_nuc - wf_correction 
e_wf_emb = e_wf_act + e_env + two_e_cross + e_nuc - wf_correction 
print("Component contributions")
print(f"{e_vqe_act}, {e_wf_act}")
print(f"{e_env}, {two_e_cross}, {wf_correction}, {dm_correction}, {e_nuc}\n")

print(f'H_emb described using {one_body_coefficients.shape[0]} qubits')
print(f'active space approx used: {active_space_approx} \n')
# Print out the final value.
print(f"FCI Energy:\t\t\t\t{expected_energy:.4f}")

print(f"Used MOs={len(active_indices)}")
print(f"Act MOs={n_act_mos}")
print(f"Env MOs={n_env_mos}")
print(f"Total MOs={len(ks.mo_occ)}\n")

print(f"DFT Energy:\t\t\t\t{e_initial:.4f}")
print(f"Error:\t\t\t\t\t{(expected_energy-e_initial)*100/expected_energy:.2f}%\n")

print(f"WF Embedding Energy:\t\t\t{e_wf_emb:.4f}")
print(f"Error:\t\t\t\t\t{(expected_energy-e_wf_emb)*100/expected_energy:.2f}%\n")

print(f"VQE H_emb min eig (via lin alg):\t{e_vqe_emb:.4f}")
print(f"Error:\t\t\t\t\t{(expected_energy-e_vqe_emb)*100/expected_energy:.2f}%\n")

if explicit_e_vqe_act:
    explicit_e_vqe_emb = explicit_e_vqe_act + e_env + two_e_cross + e_nuc - wf_correction 
    print(f"Explicit VQE Embedding Energy:\t\t{explicit_e_vqe_emb:.4f}")
    print(f"Error:\t\t\t\t\t{(expected_energy-explicit_e_vqe_emb)*100/expected_energy:.2f}%")


Component contributions
-72.01256812072756, -72.01254334979606
-11.33703574566101, 9.715146330041323, 9.723526901565492, 0.0987452587523876, 9.285714221677825

H_emb described using 12 qubits
active space approx used: False 

FCI Energy:				-75.0091
Used MOs=6
Act MOs=4
Env MOs=1
Total MOs=7

DFT Energy:				-66.5930
Error:					11.22%

WF Embedding Energy:			-74.0722
Error:					1.25%

VQE H_emb min eig (via lin alg):	-74.0723
Error:					1.25%

